#### Inputs de Cron Ejecucion

In [12]:
import pandas as pd 
import datetime as dt 
import os 
from wmf import wmf
from multiprocessing import Pool
import numpy as np
import pickle 
import alarmas as al
import glob 
import time
import pylab as pl

Bits of Cron Ejecucion

In [13]:
# Texto Fecha: el texto de fecha que se usa para guardar algunos archivos de figuras.
daten = dt.datetime.now()
dateText = dt.datetime.now().strftime('%Y-%m-%d-%H:%M')

# print '\n'
# print '###################################### Fecha de Ejecucion: '+dateText+' #############################\n'

#Obtiene las rutas necesarias 
ruta_de_rutas = '/media/nicolas/Home/Jupyter/Soraya/Op_Alarmas/Op_AMVA60m/Rutas.md'
RutasList = al.get_rutesList(ruta_de_rutas)

# rutas de objetos de entrada
ruta_cuenca = al.get_ruta(RutasList, 'ruta_cuenca')
ruta_campos = al.get_ruta(RutasList, 'ruta_campos')
ruta_codigos = al.get_ruta(RutasList, 'ruta_codigos')
ruta_configuracion_1 = al.get_ruta(RutasList, 'ruta_configuracion_1')
ruta_almacenamiento = al.get_ruta(RutasList, 'ruta_almacenamiento')
# Rutas de objetos de salida
ruta_out_rain = al.get_ruta(RutasList, 'ruta_rain')
ruta_out_rain_png = al.get_ruta(RutasList, 'ruta_rain_png')

Model_Ejec

In [14]:
#!/usr/bin/env python
from wmf import wmf 
import argparse
import textwrap
import os 
import alarmas as al
from multiprocessing import Pool
import glob
import pandas as pd 
import numpy as np 

In [15]:
cuenca=ruta_cuenca
rutaConfig=ruta_configuracion_1
verbose=True

In [16]:
def model_warper(L):
    #Ejecucion del modelo
    Res = cu.run_shia(L[1],L[2],L[3],L[4], 
        StorageLoc = L[5], ruta_storage=L[6], kinematicN=12,QsimDataFrame=False)
    #Escribe resultados 
    rutaqsim = ruta_Qsim +'_caudal/'+ QsimName +'_'+L[0].replace(' ','_').replace('-','')+'.msg'
    Qsim = pd.DataFrame(Res['Qsim'][1:].T, 
        index=Rain.index, 
        columns=posControl)
    try:
        Qsim.to_msgpack(rutaqsim)
    except:
        'no sirve'
    # 	#Actualiza historico de caudales simulados
    # 	rutaqhist = ruta_Qsim +'CaudalHistorico/'+ QsimName +'_'+L[0].replace(' ','_').replace('-','')+'_hist.msg'
    # 	al.model_write_qsim(rutaqsim,rutaqhist,posControl)
    # 	#Se actualizan los historicos de humedad de la parametrizacion asociada.
    # 	al.model_write_Stosim(L[6][:-7]+'.StOhdr',L[7])
    # 	#imprime que ya ejecuto
    if verbose: #args.
        print L[0]+' ejecutado'
    return Res

In [17]:
#lee todos los argumentos
# args=parser.parse_args()

############################ CONFIGURACION DE LA CUENCA #############################

#Lee el archivo de configuracion
ListConfig = al.get_rutesList(rutaConfig)#args.
#informacion de la lluvia
rutaRain = al.get_ruta(ListConfig, 'ruta_rainFile')
rain_bin, rain_hdr = wmf.__Add_hdr_bin_2route__(rutaRain)
DataRain = wmf.read_rain_struct(rain_hdr)
Rain = wmf.read_mean_rain(rain_hdr)
#Lee archivo de configuracion y la cuenca, almacenamiento 
cu = wmf.SimuBasin(rute=cuenca, SimSlides = True)#args.
ruta_sto = al.get_ruta(ListConfig, 'ruta_almsim')
ruta_stohist = al.get_ruta(ListConfig, 'ruta_almhist')
ruta_out_sto = al.get_ruta(ListConfig, 'ruta_out_alm')
ruta_out_slides = al.get_ruta(ListConfig, 'ruta_slides')
ruta_slides_bin, ruta_slides_hdr = wmf.__Add_hdr_bin_2route__(ruta_out_slides)
#Nombre de las simulaciones de caudal 
QsimName = al.get_ruta(ListConfig,'Qsim Name')
ruta_Qsim = al.get_ruta(ListConfig, 'ruta_qsim')
ruta_qsim_h = al.get_ruta(ListConfig, 'ruta_qsim_hist')

#Set por defecto de la modelacion
wmf.models.show_storage = 1
wmf.models.separate_fluxes = 1
wmf.models.dt = 300
wmf.models.sl_fs = 0.5
wmf.models.sim_slides = 1
posControl = wmf.models.control[wmf.models.control<>0]


#Param de configuracion
Lparam = ['Dt[seg]','Dx[mts]',
	'Almacenamiento medio',
	'Separar Flujos',
	'ruta_almacenamiento',
	'Retorno',
	'Simular Deslizamientos',
	'Factor de Seguridad FS',
	'Factor Corrector Zg']
DictParam = {}
for i in Lparam:
	a = al.get_ruta(ListConfig, i)
	DictParam.update({i:a})

#Nueva configuracion
#Prepara el tiempo
wmf.models.dt = float(DictParam['Dt[seg]'])
wmf.models.retorno = float(DictParam['Retorno'])
# Prepara los que son binarios (1) si (0) no
if DictParam['Almacenamiento medio'] == 'True':
	wmf.models.show_storage = 1
if DictParam['Separar Flujos'] == 'True':
	wmf.models.separate_fluxes = 1
if DictParam['Simular Deslizamientos'] == 'True':
	wmf.models.sim_slides = 1
	wmf.models.sl_fs = float(DictParam['Factor de Seguridad FS'])
	cu.set_Slides(wmf.models.sl_zs * float(DictParam['Factor Corrector Zg']), 'Zs')
print wmf.models.sl_zs.mean()

############################ LECTURA DE CALIBRACIONES #############################

#Lista de calibraciones
DictCalib = al.get_modelConfig_lines(ListConfig, '-c', 'Calib')
DictStore = al.get_modelConfig_lines(ListConfig, '-s', 'Store')

############################ EJECUCION ###########################################

#Prepara las ejecuciones
ListEjecs = []
Npasos = DataRain[u' Record'].shape[0]
for i in DictStore.keys():
	#trata de leer el almacenamiento 
	FileName = glob.glob(ruta_sto + DictStore[i]['Nombre'])
	if len(FileName):
		S = wmf.models.read_float_basin_ncol(ruta_sto+DictStore[i]['Nombre'],1,cu.ncells,5)[0]
	else:
		#no tiene sentido que las par que se pusieron corran sin sus C.I.
		#~ S = al.model_get_constStorage(List, cu.ncells)
		print 'Error: No se leyeron los binarios de almacenamiento operacionales.'
	#Arma la ejecucion
	Calib = DictCalib[DictStore[i]['Calib']]
	ListEjecs.append([i, Calib, rain_bin, Npasos, 1, S, ruta_sto+DictStore[i]['Nombre'],ruta_stohist+DictStore[i]['Nombre'][:-7]+'hist.msg'])

4.9948


In [20]:
lol=ListEjecs[2]

In [24]:
ListEjecs[2]

['-s 001',
 [0.001, 150.0, 1.0, 0.0, 0.6, 4.0, 0.3, 0.99, 1.0, 1.0],
 '/media/nicolas/Home/Jupyter/Soraya/Op_Alarmas/Op_AMVA60m/03_Resultados/01_rain/Lluvia_actual.bin',
 14,
 1,
 array([[  3.79545361e-01,   0.00000000e+00,   0.00000000e+00, ...,
           6.48877203e-01,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  8.27264099e+02,   7.65037231e+02,   7.65037231e+02, ...,
           9.54792786e+02,   9.79906860e+02,   1.06127771e+03],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   1.84643555e+01]], dtype=float32),
 '/media/nicolas/Home/Jupyter/Soraya/Op_Alarmas/Op_AMVA60m/03_Resultados/03_almacenamiento/CuBarbosa_001_001.StObin',
 '/media/nicolas/Home/Jupy

In [16]:
def model_warper(L):
    #Ejecucion del modelo
    Res = cu.run_shia(L[1],L[2],L[3],L[4], 
        StorageLoc = L[5], ruta_storage=L[6], kinematicN=12,QsimDataFrame=False)
    #Escribe resultados 
    rutaqsim = ruta_Qsim +'_caudal/'+ QsimName +'_'+L[0].replace(' ','_').replace('-','')+'.msg'
    Qsim = pd.DataFrame(Res['Qsim'][1:].T, 
        index=Rain.index, 
        columns=posControl)
    try:
        Qsim.to_msgpack(rutaqsim)
    except:
        'no sirve'
    # 	#Actualiza historico de caudales simulados
    # 	rutaqhist = ruta_Qsim +'CaudalHistorico/'+ QsimName +'_'+L[0].replace(' ','_').replace('-','')+'_hist.msg'
    # 	al.model_write_qsim(rutaqsim,rutaqhist,posControl)
    # 	#Se actualizan los historicos de humedad de la parametrizacion asociada.
    # 	al.model_write_Stosim(L[6][:-7]+'.StOhdr',L[7])
    # 	#imprime que ya ejecuto
    if verbose: #args.
        print L[0]+' ejecutado'
    return Res

In [28]:
Res=model_warper(lol)

-s 001 ejecutado


In [25]:
L=lol

In [26]:
rutaqsim = ruta_Qsim +'_caudal/'+ QsimName +'_'+L[0].replace(' ','_').replace('-','')+'.msg'

In [27]:
rutaqsim

'/media/nicolas/Home/Jupyter/Soraya/Op_Alarmas/Op_AMVA60m/03_Resultados/02_caudal/Qsim_Rain_s_001.msg'